In [ ]:
import random as rd
import numpy as np
import sys 

In [ ]:
def subsets(L, density):
    subset = [i for i in range(L) if rd.random() < density]
    binary = [1 if i in subset else 0 for i in range(L)]
    return subset, binary

In [ ]:
def minhash(M, hash_value_list):
    length = min(M.shape[0], hash_value_list.shape[0])
    SIG = np.full((hash_value_list.shape[1], M.shape[1]), np.inf)
    for row_idx in range(length):
        s_row_vector = M[row_idx]
        h_row_vector = hash_value_list[row_idx]
        for s_idx, s_num in enumerate(s_row_vector):
            if s_num == 1:
                SIG[:, s_idx] = np.minimum(SIG[:, s_idx], h_row_vector)
    return SIG


In [ ]:
def jaccard(a, b):
    a = set(a)
    b = set(b)
    return len(a & b) / len(a | b)


In [ ]:
def SIG_similarity(SIG, i, j):
    return (SIG[i] == SIG[j]).mean()


# the much bigger example

In [ ]:
bignum = 10000
is_prime = [True for _ in range(bignum+1)]
is_prime[0] = False

for i in range(2, bignum+1):
    if is_prime[i-1]:
        j = 2 * i
        while j <= bignum:
            is_prime[j-1] = False
            j += i
table = [i for i in range(1, bignum+1) if is_prime[i-1]]
L = table[-1]
S = [i for i in range(L)]
Q = 500
subset_list = []
M = []
seen = set()
n = table[30]
density = 0.1


created_subset_cnt = 0
while created_subset_cnt < Q:
    subset, binary = subsets(L, density)
    subset = tuple(subset)
    if subset not in seen:
        created_subset_cnt += 1
        seen.add(subset)
        subset_list.append(subset)
        M.append(binary)

M = np.array(M).transpose()

hash_value_list = []
for i in range(n):
    if i == 0:
        prime = 1
    else:
        prime = table[i-1]
    hash_value = []
    for x in range(L):
        hash_value.append((prime*x+2) % L)
    hash_value_list.append(hash_value)
hash_value_list = np.array(hash_value_list).transpose()


SIG = minhash(M, hash_value_list)
SIG = SIG.transpose()


random_list = [i for i in range(Q)]
rd.shuffle(random_list)
random_i = random_list[0]
random_j = random_list[1]
if random_i > random_j:
    random_i, random_j = random_j, random_i

subset_i = subset_list[random_i]
subset_j = subset_list[random_j]

SIG_approximated_similarity = SIG_similarity(SIG, random_i, random_j)
print("SIG approxiated Similarity of subset_{0} and subset_{1} is {2:.5f}".format(random_i, random_j, SIG_approximated_similarity))

jaccard_similarity = jaccard(subset_i, subset_j)
print("Jaccard Similarity of subset_{0} and subset_{1} is {2:.5f}".format(random_i, random_j, jaccard_similarity))

# the example from class

In [ ]:
bignum = 5
is_prime = [True for _ in range(bignum+1)]
is_prime[0] = False

for i in range(2, bignum+1):
    if is_prime[i-1]:
        j = 2 * i
        while j <= bignum:
            is_prime[j-1] = False
            j += i
table = [i for i in range(1, bignum+1) if is_prime[i-1]]
L = table[-1]
S = [i for i in range(L)]
Q = 3
subset_list = []
M = []
seen = set()
n = table[0]

created_subset_cnt = 0

subset_list.append([0,2,4])
M.append([1,0,1,0,1])
subset_list.append([2,3])
M.append([0,0,1,1,0])
subset_list.append([0,1,3])
M.append([1,1,0,1,0])

M = np.array(M).transpose()
print(M)


hash_value_list = []
for i in range(n):
    if i == 0:
        prime = 1
    else:
        prime = table[i]
    hash_value = []
    for x in range(L):
        hash_value.append((prime*x+1) % L)
    hash_value_list.append(hash_value)
hash_value_list = np.array(hash_value_list).transpose()

SIG = minhash(M, hash_value_list)
SIG = SIG.transpose()

random_list = [i for i in range(Q)]
rd.shuffle(random_list)
random_i = 0
random_j = 2
if random_i > random_j:
    random_i, random_j = random_j, random_i
subset_i = subset_list[random_i]
subset_j = subset_list[random_j]

SIG_approximated_similarity = SIG_similarity(SIG, random_i, random_j)
print("SIG approxiated Similarity of subset_{0} and subset_{1} is {2:.1f}".format(random_i, random_j, SIG_approximated_similarity))

jaccard_similarity = jaccard(subset_i, subset_j)
print("Jaccard Similarity of subset_{0} and subset_{1} is {2:.1f}".format(random_i, random_j, jaccard_similarity))